In [35]:
import numpy as np
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import csv
from datetime import datetime
import glob

In [36]:
#Get APFS Numbers of the current APFS listing. Save them as 'today_ids'

today_df = pd.read_csv('today_apfs.csv')
today_df = today_df.fillna('Not provided')
today_ids = pd.DataFrame(today_df[('APFS Number')])
today_ids

,APFS Number
0,F2019046845
1,F2019048087
2,F2020048899
3,F2020048969
4,F2020049267
5,F2020049269
6,F2020049846
7,F2020049880
8,F2020050067
9,F2020050098


In [37]:
#Get APFS Numbers of the previous APFS listing. Save them as 'old_ids'

old_df = pd.read_csv('jan_apfs.csv')
old_df = old_df.fillna('Not provided')
old_ids = pd.DataFrame(old_df[('APFS Number')])
old_ids

,APFS Number
0,F2016032728
1,F2018042285
2,F2019046187
3,F2019046571
4,F2019046792
5,F2019046845
6,F2019047413
7,F2019048104
8,F2019048283
9,F2019048288


In [38]:
# Do a left merge on today_ids with old_ids. Save the dataframe as df_merged_ids

df_merged_ids = old_ids.merge(today_ids, on=['APFS Number'], 
                   how='left', indicator=True)
df_merged_ids

,APFS Number,_merge
0,F2016032728,left_only
1,F2018042285,left_only
2,F2019046187,left_only
3,F2019046571,left_only
4,F2019046792,left_only
5,F2019046845,both
6,F2019047413,left_only
7,F2019048104,left_only
8,F2019048283,left_only
9,F2019048288,left_only


In [39]:
# APFS Numbers in df_new_ids that have the value 'left_only' for '_merge' are the new IDs. Drop the others and save as df_new_ids

df_old_ids = df_merged_ids[~df_merged_ids['_merge'].isin(['both'])]
df_old_ids

,APFS Number,_merge
0,F2016032728,left_only
1,F2018042285,left_only
2,F2019046187,left_only
3,F2019046571,left_only
4,F2019046792,left_only
6,F2019047413,left_only
7,F2019048104,left_only
8,F2019048283,left_only
9,F2019048288,left_only
10,F2019048492,left_only


In [40]:
# Merge our today_df with df_new_ids to get all information on new APFS numbers

removed_today_df = df_old_ids.merge(old_df, on=['APFS Number'], 
                   how='left')
del removed_today_df['_merge']
removed_today_df

,APFS Number,Description,Contract Status,Component,NAICS code,Small Business Program,Dollar Range,Contract Vehicle,Contractor,Contract Number,Contract Complete,Estimated Release,Anticipated Award Quarter,POC Name,POC Phone,POC Email
0,F2016032728,The Contractor shall provide services to suppo...,NEW,ICE/CIO,541519,SB,"$2,000,000.00 to $5,000,000.00",GWAC/MAC,Not provided,Not provided,2025-02-27,2020-01-15,Q2 2020,Phyllis Brokenborough,202-732-2381,phyllis.brokenborough@ice.dhs.gov
1,F2018042285,The contractor shall provide SAMS management c...,REC,ICE/CIO,541519,SB,"$250,000.00 to $500,000.00",GSA,Annams Systems Corporation,GS-35F-0504L/HSCETC-15-F-00015,2021-08-15,2018-07-09,Q4 2021,Jason Bush,202-732-2839,jason.bush@ice.dhs.gov
2,F2019046187,"The Department of Homeland Security (DHS), U.S...",NEW,ICE/ERO,561612,None,"Over $100,000,000.00",Contract,Not provided,Not provided,2029-04-19,2019-09-30,Q2 2020,OAQ DCR,202-732-2600,CaliforniaDetention@ice.dhs.gov
3,F2019046571,Immigration and Customs Enforcement (ICE) seek...,NEW,ICE/AS,921190,SB,"$500,000.00 to $1,000,000.00",Contract,Not provided,Not provided,2025-02-28,2020-01-17,Q2 2020,Jamaal Robinson,202-732-6233,jamaal.s.robinson@ice.dhs.gov
4,F2019046792,The objective of this contract is to provide c...,REC,ICE/ERO,561210,None,"$20,000,000.00 to $50,000,000.00",GSA,Nakamoto Group Inc.,HSCECR-15-F-00008,2024-10-31,2019-11-18,Q2 2020,Sheneil Green,202-732-2834,sheneil.green@ice.dhs.gov
5,F2019047413,ICE seeks air charter services in order to con...,REC,ICE/ERO,481211,SB,"$50,000,000.00 to $100,000,000.00",GSA,ATS,HSCECR-17-F-00001,2025-07-31,2020-01-06,Q3 2020,LeeAnna McGinn,2025679018,leeanna.mcginn@ice.dhs.gov
6,F2019048104,The purpose of this requirement is to obtain c...,REC,ICE/ERO,561990,None,"$20,000,000.00 to $50,000,000.00",DWAC/OASIS,Booz Allen Hamilton,HSCEDM15F00013,2025-03-15,2019-12-20,Q2 2020,Scott Lorick,2027324522,Scott.R.Lorick@ice.dhs.gov
7,F2019048283,Acquisition Management Support for the Student...,NEW,ICE/HSI,541611,None,"$5,000,000.00 to $10,000,000.00",GWAC/MAC,Not provided,Not provided,2021-02-09,2019-12-12,Q2 2020,Shereen Demarais,214-905-8317,michelle.d.kelley@ice.dhs.gov
8,F2019048288,The requirement is to provide training and sof...,NEW,ICE/AS,541611,SB,"$250,000.00 to $500,000.00",Contract,Not provided,Not provided,2025-02-28,2020-01-03,Q2 2020,Maricela Perez,214-902-5130,Maricela.Perez@ice.dhs.gov
9,F2019048492,The objective of the eHR O&M Support Service C...,REC,ICE/CIO,518210,SB,"$10,000,000.00 to $20,000,000.00",DWAC/EAGLE II,306 IT Integrated Solutions,HSHQDC-13-D-E2066 / HSCETC-14-J-00117,2021-02-28,2020-01-10,Q2 2020,Christie Harris,202-732-2396,christie.l.harris@ice.dhs.gov


In [41]:
removed_today_summary = removed_today_df[['APFS Number', 'Description']]
removed_today_summary = removed_today_summary.set_index('APFS Number')
removed_today_summary

,Description
APFS Number,
F2016032728,The Contractor shall provide services to suppo...
F2018042285,The contractor shall provide SAMS management c...
F2019046187,"The Department of Homeland Security (DHS), U.S..."
F2019046571,Immigration and Customs Enforcement (ICE) seek...
F2019046792,The objective of this contract is to provide c...
F2019047413,ICE seeks air charter services in order to con...
F2019048104,The purpose of this requirement is to obtain c...
F2019048283,Acquisition Management Support for the Student...
F2019048288,The requirement is to provide training and sof...
